In [1]:
import numpy as np
import pandas as pd

# Data Import

In [2]:
path = '/Users/rackhun/Desktop/Python/deep_learning/GDG/RNN_study/titanic/data/'

train_file='train.csv'
test_file = 'test.csv'

In [3]:
train_data = pd.read_csv(path+train_file)
test_data = pd.read_csv(path+test_file)

In [4]:
# dummies
dum_train = train_data
dum_test = test_data

In [5]:
label = pd.DataFrame({'label': dum_train['Survived']})
del dum_train['Survived']

In [6]:
all_df = pd.concat([dum_train, dum_test], axis=0).reset_index(drop=True)

print(len(train_data),'+', len(test_data), '=', len(train_data)+len(test_data))
print('result = ',len(all_df))

891 + 418 = 1309
result =  1309


# Null-values

In [7]:
print(all_df.isnull().sum())

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64


# Name

In [8]:
def name_class(data):
    name_list=[]
    for i in range(len(data['Name'])):
        test=list(filter(lambda ch: ch=='Mr.' or     
                                             ch=='Mrs.' or         
                                             ch=='Miss.' or           
                                             ch=='Master.' or       #  young man, boys (has dominance)                      
                                             ch=='Don.' or           # Spanish male ( higer-ranking: professors)   # no in test
                                             ch=='Dona.' or          # Spanish female    # only in test
                                             ch=='Rev.' or            # members of Christian clergy     -> 0%
                                             ch=='Dr.' or               # Doctors or phd.
                                             ch=='Ms.' or              # female (Mrs. or Miss.) 
                                             ch=='Major.' or          # higher ranking than captain (Col. > Major. > Capt.)       # no in test
                                             ch=='Lady.' or           # British female (wife of knight or military, noble)           # no in test  -> 100%
                                             ch=='Sir.' or               # British Man (noble or rich)                                            # no in test   -> 100%
                                             ch=='Mlle.' or             # Mademoiselle : woman employed to teach children in private households     # no in test
                                             ch=='Col.' or               # Colonel : high ranking in the army or air-force
                                             ch=='Capt.' or             # Caption     # no in test   -> 0%
                                             ch=='Mme.' or           # Madame : French for Mrs.      # no in test   -> 100%
                                             ch=='Countess.' or     # wife or widow of nobleman (noble)     # no in test   -> 100%
                                             ch=='Jonkheer.',         #  Dutch female lowest nobility     # no in test     ->0%
                                             data['Name'][i].split()))
        if not test:
            print(i)
        else:
            name_list.append(test[0])
            
    return name_list

In [9]:
all_df['Name'] = name_class(all_df)

In [10]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,Mr.,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,Mrs.,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,Miss.,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,Mrs.,female,35.0,1,0,113803,53.1000,C123,S
4,5,3,Mr.,male,35.0,0,0,373450,8.0500,NaN,S


# Cabin

In [11]:
def isNaN(num):
    return num != num

In [12]:
def cabin_class(data):
    cabin_list=[]
    for i in range(len(data['Cabin'])):
        
        if isNaN(data['Cabin'][i]):
            cabin_list.append('NaN')
        else:
            cabin_list.append(list(data['Cabin'][i])[0])
            
    return cabin_list

In [13]:
all_df['Cabin'] = cabin_class(all_df)

# Ticket

In [14]:
# version 1, 2
#del all_df['Ticket'] 

In [15]:
# version 3
def ticket_class(data):
    ticket_list = []
    numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    for i in range(len(all_df['Ticket'])):
        letters=[]
    
        if any( np.array(numbers) == all_df['Ticket'][i][0] ):
            ticket_list.append('Number')
            continue
    
        for letter in all_df['Ticket'][i]:
            if letter != ' ':
                letters.append(letter)
            else:
                break
    
        ticket_list.append(''.join(letters))
        
    return ticket_list

In [16]:
print(all_df['Ticket'][2])
print(all_df['Ticket'][115])

STON/O2. 3101282
STON/O 2. 3101294


In [20]:
# version 4
def ticket_class2(data):
    ticket_letter_list = []
    ticket_number_list = []
    numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    
    for i in range(len(all_df['Ticket'])):
        
        ### rectify  'STON/O 2.'  => 'STON/O2.'
        if all_df['Ticket'][i][:9] == 'STON/O 2.':
            all_df['Ticket'][i] = 'STON/O2. '+ all_df['Ticket'][i][10:]
            
        ### rectify  'SC/AH Basle'  => 'SC/AH_Basle'
        #if all_df['Ticket'][i][:11] == 'SC/AH Basle':
            #all_df['Ticket'][i] = 'SC/AH_Basle '+ all_df['Ticket'][i][12:]
        #####
    
        if any( np.array(numbers) == all_df['Ticket'][i][0] ):
            ticket_letter_list.append('Number')
            ticket_number_list.append( float(all_df['Ticket'][i]) )
        
        elif all_df['Ticket'][i] == 'LINE':
            ticket_letter_list.append( all_df['Ticket'][i] )
            ticket_number_list.append(float(0))
            
        else:
            for n, letter in enumerate(reversed(all_df['Ticket'][i])):
                if letter == ' ':
                    ticket_letter_list.append( all_df['Ticket'][i][:-(n+1)] )
                    ticket_number_list.append( float(all_df['Ticket'][i][-n:]) )
                    break   # only for the last ' '
        
    return ticket_letter_list, ticket_number_list

In [21]:
ticket_letter, ticket_number = ticket_class2(all_df)

all_df['Ticket_letter'] = pd.Series(ticket_letter, index=all_df.index)
all_df['Ticket_number'] = pd.Series(ticket_number, index=all_df.index)

In [22]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_letter,Ticket_number
0,1,3,Mr.,male,22.0,1,0,A/5 21171,7.2500,NaN,S,A/5,21171.0
1,2,1,Mrs.,female,38.0,1,0,PC 17599,71.2833,C,C,PC,17599.0
2,3,3,Miss.,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,STON/O2.,3101282.0
3,4,1,Mrs.,female,35.0,1,0,113803,53.1000,C,S,Number,113803.0
4,5,3,Mr.,male,35.0,0,0,373450,8.0500,NaN,S,Number,373450.0


# Age & Fare (Null-values)

In [23]:
#means_age = all_df.groupby(['Name', 'Sex', 'Pclass'])['Age'].mean()
means_age = all_df.groupby(['Name', 'Sex'])['Age'].mean()
means_fare = all_df.groupby(['Name', 'Sex', 'Pclass', 'Embarked'])['Fare'].mean()

In [24]:
def age_fare(data):
    
    for i in range(len(data['Age'])):
        
        if isNaN(data['Age'][i]):
            #data['Age'][i] = means_age[data['Name'][i], data['Sex'][i], data['Pclass'][i]]
            #data['Age'][i] = means_age[data['Name'][i], data['Sex'][i]]
            data['Age'][i] = means_age[data['Name'][i], data['Sex'][i]]
            
        if isNaN(data['Fare'][i]):
            data['Fare'][i] = means_fare[data['Name'][i], data['Sex'][i], data['Pclass'][i], data['Embarked'][i]]
            #data['Fare'][i] = means_fare[data['Name'][i], data['Sex'][i]]
            
    return data

In [25]:
all_df = age_fare(all_df)

/Users/rackhun/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/rackhun/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
print(all_df.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         2
Ticket_letter    0
Ticket_number    0
dtype: int64


# Embarked (Null-values)

In [27]:
frequency_embarked = all_df.groupby(['Name', 'Sex', 'Pclass'])['Embarked'].value_counts()

In [28]:
def embarked(data):
    
    for i in range(len(data['Embarked'])):
        
        if isNaN(data['Embarked'][i]):
            data['Embarked'][i] = frequency_embarked[data['Name'][i], data['Sex'][i], data['Pclass'][i]].idxmax()
            #data['Embarked'][i] = frequency_embarked[data['Name'][i], data['Sex'][i]].idxmax()

    return data

In [29]:
all_df = embarked(all_df)

/Users/rackhun/miniconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [30]:
print(all_df.isnull().sum())

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
Ticket_letter    0
Ticket_number    0
dtype: int64


# Categorical Data

In [31]:
all_df = all_df.merge(pd.get_dummies(all_df["Pclass"], prefix="pclass"), left_index=True, right_index=True)
all_df = all_df.merge(pd.get_dummies(all_df["Name"], prefix="name"), left_index=True, right_index=True)
all_df = all_df.merge(pd.get_dummies(all_df["Sex"], prefix="sex"), left_index=True, right_index=True)
all_df = all_df.merge(pd.get_dummies(all_df["SibSp"], prefix="sibsp"), left_index=True, right_index=True)   # version 2
all_df = all_df.merge(pd.get_dummies(all_df["Parch"], prefix="parch"), left_index=True, right_index=True)   # version 2
#all_df = all_df.merge(pd.get_dummies(all_df["Ticket"], prefix="ticket"), left_index=True, right_index=True)   # version 3 
all_df = all_df.merge(pd.get_dummies(all_df["Ticket_letter"], prefix="ticket_letter"), left_index=True, right_index=True)   # version 4
all_df = all_df.merge(pd.get_dummies(all_df["Cabin"], prefix="cabin"), left_index=True, right_index=True)
all_df = all_df.merge(pd.get_dummies(all_df["Embarked"], prefix="embarked"), left_index=True, right_index=True)

In [32]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_NaN,cabin_T,embarked_C,embarked_Q,embarked_S
0,1,3,Mr.,male,22.0,1,0,A/5 21171,7.2500,NaN,...,0,0,0,0,0,1,0,0,0,1
1,2,1,Mrs.,female,38.0,1,0,PC 17599,71.2833,C,...,1,0,0,0,0,0,0,1,0,0
2,3,3,Miss.,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,...,0,0,0,0,0,1,0,0,0,1
3,4,1,Mrs.,female,35.0,1,0,113803,53.1000,C,...,1,0,0,0,0,0,0,0,0,1
4,5,3,Mr.,male,35.0,0,0,373450,8.0500,NaN,...,0,0,0,0,0,1,0,0,0,1


In [33]:
del all_df['Pclass']
del all_df['Name']
del all_df['Sex']
del all_df['SibSp']
del all_df['Parch']
del all_df['Ticket'] 
del all_df['Ticket_letter']  # version 4
del all_df['Cabin']
del all_df['Embarked']

In [34]:
all_df.head()

,PassengerId,Age,Fare,Ticket_number,pclass_1,pclass_2,pclass_3,name_Capt.,name_Col.,name_Countess.,...,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_NaN,cabin_T,embarked_C,embarked_Q,embarked_S
0,1,22.0,7.2500,21171.0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,2,38.0,71.2833,17599.0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,26.0,7.9250,3101282.0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,4,35.0,53.1000,113803.0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,5,35.0,8.0500,373450.0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


# Normalize Continuous Data

In [35]:
def normalize(data):
    return (data - data.mean()) / data.std()

In [36]:
all_df['Age'] = normalize(all_df['Age'])
all_df['Fare'] = normalize(all_df['Fare'])
all_df['Ticket_number'] = normalize(all_df['Ticket_number'])

# Split Data ( Train / Test )

In [37]:
train = all_df[:len(train_data)]
test = all_df[len(train_data):].reset_index(drop=True)

In [38]:
train.head()

,PassengerId,Age,Fare,Ticket_number,pclass_1,pclass_2,pclass_3,name_Capt.,name_Col.,name_Countess.,...,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_NaN,cabin_T,embarked_C,embarked_Q,embarked_S
0,1,-0.598510,-0.503051,-0.412185,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,2,0.614245,0.734492,-0.417807,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,3,-0.295321,-0.490005,4.435373,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,4,0.386853,0.383071,-0.266399,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
4,5,0.386853,-0.487590,0.142240,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [39]:
test.head()

,PassengerId,Age,Fare,Ticket_number,pclass_1,pclass_2,pclass_3,name_Capt.,name_Col.,name_Countess.,...,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_NaN,cabin_T,embarked_C,embarked_Q,embarked_S
0,892,0.348955,-0.491857,0.075291,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,893,1.296419,-0.507882,0.126222,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,894,2.433376,-0.455942,-0.067352,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,895,-0.219524,-0.475752,0.050493,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,896,-0.598510,-0.405693,4.435398,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1


# Write results

In [40]:
train.to_csv("./data_preprocess/train_preprocess4.csv", index=False)
test.to_csv("./data_preprocess/test_preprocess4.csv", index=False)
label.to_csv("./data_preprocess/label_preprocess4.csv", index=False)